In [68]:
# Prime Seive
import math

In [212]:
def prime_sieve(n=30):
    candidates = [i for i in range(2, n+1)]
    crossed_off = []
    limit = int(math.sqrt(n))
    for current_index in range(limit+1): 
        step = candidates[current_index]
        if step not in crossed_off:
            for i in range(current_index+step, len(candidates), step):              
                if not candidates[i] in crossed_off:
                    crossed_off.append(candidates[i])
    
    return [i for i in candidates if i not in crossed_off]        

In [ ]:
set(candidates).symmetric_difference(set(crossed_out))

In [326]:
def prime_sieve5(n=30):
    candidates = [i for i in range(2, n+1)]
    crossed_off = []
    limit = int(math.sqrt(n))
    for current_index in range(limit+1): 
        step = candidates[current_index]
        if step not in crossed_off:
            for i in range(current_index+step, len(candidates), step):              
                if not candidates[i] in crossed_off:
                    crossed_off.append(candidates[i])
    
    return set(candidates).symmetric_difference(set(crossed_off))

In [331]:
def prime_sieve6(n=30):
    candidates = [i for i in range(2, n+1)]
    crossed_off = set()
    limit = int(math.sqrt(n))
    for current_index in range(limit+1): 
        step = candidates[current_index]
        if step not in crossed_off:
            for i in range(current_index+step, len(candidates), step):              
                if not candidates[i] in crossed_off:
                    crossed_off.add(candidates[i])
    
    return set(candidates).symmetric_difference(crossed_off)

In [213]:
#answers up to 30
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29]

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29]

In [214]:
prime_counts = { 10 : 4,                 # Historical data for validating our results - the number of primes
                 100 : 25,               # to be found under some limit, such as 168 primes under 1000
                 1000 : 168,
                 10000 : 1229,
                 100000 : 9592,
                 1000000 : 78498,
                 10000000 : 664579,
                 100000000 : 5761455
                }

In [215]:
len(prime_sieve(10))

4

In [216]:
def prime_sieve2(n=30):
    candidates = [True] * (n-1)
    limit = int(math.sqrt(n))    
    for step in range(2, limit+1): 
        # if not been crossed off list
        current_index = step - 2
        if candidates[current_index]:
            for i in range(current_index+step, len(candidates), step):     
                candidates[i] = False
    
    return [i+2 for i in range(len(candidates)) if candidates[i]]        

In [288]:
def prime_sieve3(n=30):
    candidates = [True] * (n)
    candidates[0] = candidates[1] = False
    limit = int(math.sqrt(n))    
    for step in range(2, limit+1): 
        # if not been crossed off list
        current_index = step
        if candidates[current_index]:
            for i in range(current_index+step, len(candidates), step):     
                candidates[i] = False
    
    return [i for i in range(len(candidates)) if candidates[i]]        

In [289]:
N = 1000
results2 = prime_sieve2(N)
#print(results2)
print(len(results2))

168


In [290]:
results1 = prime_sieve(N)
#print(results1)
print(len(results1))

168


In [318]:
results3 = prime_sieve3(N)
#print(results2)
print(len(results3))

168


In [321]:
results4 = prime_sieve4(N)
#print(results2)
print(len(results4))

168


In [332]:
results6 = prime_sieve6(N)
#print(results2)
print(len(results6))

168


In [219]:
%timeit prime_sieve(N)

9.54 ms ± 47.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [327]:
%timeit prime_sieve5(N)

5.27 ms ± 23.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [333]:
%timeit prime_sieve6(N)

137 µs ± 1.62 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [294]:
%timeit prime_sieve2(N)

70.5 µs ± 919 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [293]:
%timeit prime_sieve3(N)

68.6 µs ± 1.31 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [323]:
%timeit prime_sieve4(N)

40.2 µs ± 469 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [295]:
%timeit prime_sieve_np(N)

10.6 µs ± 97.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [253]:
import numpy as np

In [265]:
def prime_sieve_np(n=30):
    candidates = np.ones(n, dtype=np.bool)
    limit = int(math.sqrt(n))    
    candidates[0] = candidates[1] = False
    
    for i in range(2, limit+1):
        if candidates[i]:
            candidates[i*i::i] = False
    return np.flatnonzero(candidates)

In [298]:
candidates = np.arange(30)

In [299]:
i = 2
candidates[i*i::i]


array([ 4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28])

In [302]:
candidates

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])

In [306]:
candidates[i*i :: i]

array([ 4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28])

In [269]:
len(prime_sieve_np(N))

168

In [284]:
for n, expected in list(prime_counts.items()):
    print(len(prime_sieve_np(n)), expected)

4 4
25 25
168 168
1229 1229
9592 9592
78498 78498
664579 664579
5761455 5761455


In [285]:
for n, expected in list(prime_counts.items()):
    print(len(prime_sieve2(n)), expected)

4 4
25 25
168 168
1229 1229
9592 9592
78498 78498
664579 664579
5761455 5761455


In [324]:
for n, expected in list(prime_counts.items()):
    print(len(prime_sieve4(n)), expected)

4 4
25 25
168 168
1229 1229
9592 9592
78498 78498
664579 664579
5761455 5761455


In [311]:
candidates = [i for i in range(30+1)]

In [312]:
print(candidates)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]


In [315]:
candidates[2::2] = [False] * len(candidates[2::2])

In [317]:
print(candidates)

[0, 1, False, 3, False, 5, False, 7, False, 9, False, 11, False, 13, False, 15, False, 17, False, 19, False, 21, False, 23, False, 25, False, 27, False, 29, False]


In [320]:
def prime_sieve4(n=30):
    candidates = [True] * (n)
    candidates[0] = candidates[1] = False
    limit = int(math.sqrt(n))    
    for i in range(2, limit+1): 
        # if not been crossed off list
        if candidates[i]:
            candidates[i*i::i] = [False] * len(candidates[i*i::i])
                
    return [i for i in range(len(candidates)) if candidates[i]]        